In [18]:
with open('input.txt', 'r') as f:
    raw_lines = f.readlines()

positions = [[int(c) for c in line.replace('\n', '').split(',')] for line in raw_lines]

# Part 1

In [19]:
def get_sorted_by_areas(positions):
    dmap = {}
    for (i, (x1, y1)) in enumerate(positions):
        for (j_offset, (x2, y2)) in enumerate(positions[i + 1:]):
            j = i + 1 + j_offset
            dmap[(i, j)] = abs(x1 - x2 + 1) * abs(y1 - y2 + 1)

    return sorted(dmap.keys(), key=lambda ij: dmap[ij])

In [20]:
i, j = get_sorted_by_areas(positions)[-1]
positions[i], positions[j], (positions[i][0] - positions[j][0] + 1) * (positions[i][1] - positions[j][1] + 1)

([83933, 85142], [16727, 14058], 4777409595)

# Part 2